In [1]:
import csv
from algos.helper_functions import create_file, get_query_latency
from energy.energy_functions import *
from algos.SA import simulated_annealing
algo_metadata={}
solutions = []
algo_energy = []
solution_energy = []
pg_energy = []

solution_cost = []
pg_cost = []

solution_latency = []
pg_latency = []

# Set the path to the directory containing the files
directory_path = "JOB-queries"

index = 0
# Loop over all the files in the directory
for filename in os.listdir(directory_path):
    index = index + 1
    # Check if the file is a file and not a directory
    if os.path.isfile(os.path.join(directory_path, filename)) and index < 200:  # and filename in elements:
        # Open the file for reading
        with open(os.path.join(directory_path, filename), 'r', encoding="utf8") as file:
            # Read the contents of the file
            query = file.read()
            algo_metadata[filename] = []
            psensor = findPowerSensor("YWATTMK1-1F6860.power")
            stopDataRecording(psensor)
            clearPowerMeterCache(psensor)
            tm = time.time()
            datalog = psensor.get_dataLogger()
            datalog.set_timeUTC(time.time())
            startDataRecording(psensor)  # Power Meter starts recording power per second
            time.sleep(2.0)
            print("4 - is recording: ", psensor.get_dataLogger().get_recording())
            startTime = datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')

            #apply the algorithm
            solution, cost = simulated_annealing(query, 10, 8, 0.5)
            create_file("solutions/sa", filename, solution)

            endTime = datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')
            # print("4-4 - is recording: ", psensor.get_dataLogger().get_recording())
            print("startTime: ", startTime, " - endTime: ", endTime)
            # YAPI.Sleep(2000)
            time.sleep(2.0)
            print("stop recording : ", datetime.now())
            stopDataRecording(psensor)
            print("7 - is recording: ", psensor.get_dataLogger().get_recording())

            (power, exec_time, energy) = getAveragePower(psensor, startTime, endTime)

            (query_power, query_exec_time, query_energy) = get_query_exec_energy(input, False)

            (query_power, query_exec_time, solution_query_energy) = get_query_exec_energy(solution, True)

            # solution_cost.append(cost)
            # algo_energy.append(energy)
            # solution_energy.append(solution_query_energy)
            # pg_energy.append(query_energy)
            # solutions.append(solution)
            algo_metadata[filename].append(energy)
            algo_metadata[filename].append(solution_query_energy)
            algo_metadata[filename].append(get_query_latency(solution, True))

        # print("solution_cost : ", solution_cost)
        # print("algo_energy :", algo_energy)
        # print("solution_energy", solution_energy)
        # print("pg_energy", pg_energy)
print("saving csv")
# open a CSV file for writing
with open('sa_data.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)

    # write the header row
    writer.writerow(['Filename', 'algo energy', 'Query Energy', 'Latency'])
    for filename, data in algo_metadata.items():
        writer.writerow([filename, data[0], data[1],data[2]])


YAPI_Exception: YoctoApi error : No function of that class (yapi:4434)